In [3]:
import qiskit.quantum_info as qi
import numpy as np
import pandas as pd
import copy

def Mdot(Ml):
    mtp = Ml[0]
    for i in range(1,len(Ml)):
        mtp = np.dot(mtp,Ml[i])
    return mtp

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

def print_matrix(M):
    return pd.DataFrame(np.real(M)).round(1)

In [5]:
def latex_matrix(M, r = 1):
    ltp = "\setcounter{MaxMatrixCols}{" + str(len(M)) +"} \\begin{equation} \\begin{pmatrix} "
    for V in M:
        for e in V:
            ltp = ltp +str(round(np.real(e),r)) + " & "
        ltp = ltp[:-2]
        ltp = ltp +"\\\\"
    ltp = ltp[:-2]
    ltp = ltp + "\\end{pmatrix} \\end{equation}"
    return ltp

#M = [[1,2,3],[4,5,6]]
#print(latex_matrix(M))

In [6]:
# A function to print out the binary number for N-sites
def bi(num,N):
    bi = bin(num)
    out = ""
    for i in range(2,len(bi)):
        out = out + bi[i]
    no = len(out)
    for i in range(no,N):
        out = '0' + out
    return out


# Form of various types of interaction

Here I will look at the form of various types of interactions in the basis where the spin is split and fixed.

First let us define different types of interactions.

### On-Site interaction

This is the one we currently use where the interaction is only between opposite spins on the same site.

$$ \hat{U} = u \sum_{i=0}^{N-1}\hat{n}_{i\uparrow}\hat{n}_{i\downarrow} $$

In [7]:
def O(label):
    return qi.Operator.from_label(label).data

def U_OnSite(u,N):
    label_0 = ['I' for i in range(N)]
    l0 = ''.join(label_0)
    utp = O(l0 + l0)*0
    for i in range(0,N):
        label_i = copy.copy(label_0)
        label_i[i] = 'Z'
        li = ''.join(label_i)
        n1 = 1/2*O(l0 + l0) - 1/2*O(li + l0)
        n2 = 1/2*O(l0 + l0) - 1/2*O(l0 + li)
        utp = utp + u*np.dot(n1,n2)
    return utp

In [8]:
pd.DataFrame(np.real(U_OnSite(1,2)))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Hartree Interaction

Here we have interactions between any two electrons of the same spin.  

$$ \hat{U} = \sum_{\sigma}\sum_{ij} u_{ij} \hat{n}_{i\sigma}\hat{n}_{i\sigma}$$

Let's let* $$ u_{ij} = \frac{1}{|i-j|}$$

*for periodic boundary conditions, we need to adjust this.

Alternetively, we can just keep the nearest neihgbour interaction,
$$ u_{ij} = \delta_{i,j+1} + \delta_{i+1,j} $$

In [7]:
#Coulomb
def u(i,j,N):
    return 1/abs(i-j)

#Nearest Neihgbour
def u(i,j,N):
    if j == np.mod(i+1,N):
        return 1
    elif i == np.mod(j+1,N):
        return 1
    else:
        return 0

def U_Hartree(N):
    label_0 = ['I' for i in range(N)]
    l0 = ''.join(label_0)
    utp = O(l0 + l0)*0
    for i in range(0,N):
        for j in range(i+1,N):
            label_i = copy.copy(label_0)
            label_j = copy.copy(label_0)
            label_i[i] = 'Z'
            label_j[j] = 'Z'
            li = ''.join(label_i)
            lj = ''.join(label_j)
            n1u = 1/2*O(l0 + l0) - 1/2*O(li + l0)
            n2u = 1/2*O(l0 + l0) - 1/2*O(lj + l0)
            n1d = 1/2*O(l0 + l0) - 1/2*O(l0 + li)
            n2d = 1/2*O(l0 + l0) - 1/2*O(l0 + lj)
            utp = utp + u(i,j,N)*np.dot(n1u,n2u) + u(i,j,N)*np.dot(n1d,n2d)
    return utp

In [8]:
print_matrix(U_Hartree(2))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Exchange interaction

Here we still have two electrons but there is potential for the electrons to flip their spin.

$$ \hat{U} = \sum_{\sigma \sigma'}\sum_{ij} k_{\sigma \sigma' i j} c^{\dagger}_{i \sigma} c^{\dagger}_{j \sigma'} c_{i \sigma'} c_{j \sigma} $$

Let the interaction strength be given as*
$$ F_{\sigma \sigma' ij} = \frac{1}{|i-j|+|\sigma-\sigma|} $$


*I just made this up based on k being finite for $i=j$ and $\sigma \neq \sigma$.  The real form should be checked.

Again we can do nearest neighbor instead,
$F_{i,j} = \delta_{i,j+1} + \delta_{i+1,j} $

In [11]:
#Guess
def F(s,ss,i,j,N):
    return 1/(abs(i-j)+abs(s-ss)+0.001)

#Nearest Neihgbour
def F(s,ss,i,j,N):
    if j == np.mod(i+1,N):
        return 1
    elif i == np.mod(j+1,N):
        return 1
    else:
        return 0

def c(s,i,N):
    label_0 = ['I' for i in range(N)]
    label_x = ['I' for i in range(N)]
    label_y = ['I' for i in range(N)]
    l0 = ''.join(label_0)
    for j in range(0,i):
        label_x[j] = 'Z'
        label_y[j] = 'Z'
    label_x[i] = 'X'
    label_y[i] = 'Y'
    lx = ''.join(label_x)
    ly = ''.join(label_y)
    if s == 0:
        ox = O(lx + l0)
        oy = O(ly + l0)
    elif s == 1:
        ox = O(l0 + lx)
        oy = O(l0 + ly)
    else:
        print("s = 0 for spin up and s = 1 for spin down")
    oc = 1/2*(ox + 1j*oy)
    return oc

def cd(s,i,N):
    label_0 = ['I' for i in range(N)]
    label_x = ['I' for i in range(N)]
    label_y = ['I' for i in range(N)]
    l0 = ''.join(label_0)
    for j in range(0,i):
        label_x[j] = 'Z'
        label_y[j] = 'Z'
    label_x[i] = 'X'
    label_y[i] = 'Y'
    lx = ''.join(label_x)
    ly = ''.join(label_y)
    if s == 0:
        ox = O(lx + l0)
        oy = O(ly + l0)
    elif s == 1:
        ox = O(l0 + lx)
        oy = O(l0 + ly)
    else:
        print("s = 0 for spin up and s = 1 for spin down")
    oc = 1/2*(ox - 1j*oy)
    return oc


In [12]:
#Notice that there is some cancalation
i=0
j=1
s=0
ss=0
N=2

#print_matrix(Mdot([cd(s,i,N),cd(ss,j,N),c(ss,i,N),c(s,j,N)]) )

In [13]:
def U_Exchange(N):
    label_0 = ['I' for i in range(N)]
    l0 = ''.join(label_0)
    utp = O(l0 + l0)*0
    for i in range(0,N):
        for j in range(0,N):
            for s in range(0,2):
                for ss in range(0,2):
                    cdis = cd(s,i,N)
                    cdjss = cd(ss,j,N)
                    ciss = c(ss,i,N)
                    cjs = c(s,j,N)
                    utp = utp + F(s,ss,i,j,N)*Mdot([cdis,cdjss,ciss,cjs])                                
    return utp

In [14]:
print_matrix(U_Exchange(2))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
bi(15,4)

'1111'

### Coulomb-assisted hopping

$$ U = \sum_{ij}\sum_{\sigma,\sigma'}X_{ij\sigma\sigma'}\left(c^{\dagger}_{i\sigma}c^{\dagger}_{i\sigma'}c_{i\sigma'}c_{j\sigma} + c^{\dagger}_{i\sigma}c^{\dagger}_{j\sigma'}c_{j\sigma'}c_{j\sigma} \right)$$

We can use the nearest neighbour interaction
$$X_{ij} = \delta_{i,j+1} + \delta_{i+1,j}$$

In [14]:
#Nearest Neihgbour
def X(s,ss,i,j,N):
    if j == np.mod(i+1,N):
        return 1
    elif i == np.mod(j+1,N):
        return 1
    else:
        return 0

In [15]:
def U_Assisted(N):
    label_0 = ['I' for i in range(N)]
    l0 = ''.join(label_0)
    utp = O(l0 + l0)*0
    for i in range(0,N):
        for j in range(0,N):
            for s in range(0,2):
                for ss in range(0,2):
                    cdis = cd(s,i,N)
                    cdiss = cd(ss,i,N)
                    ciss = c(ss,i,N)
                    cdjss = cd(ss,j,N)
                    cjss = c(ss,j,N)
                    cjs = c(s,j,N)
                    utp = utp + X(s,ss,i,j,N)*Mdot([cdis,cdiss,ciss,cjs])+ X(s,ss,i,j,N)*Mdot([cdis,cdjss,cjss,cjs])                                   
    return utp

In [16]:
print_matrix(U_Assisted(2))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### Simultaneous hopping

$$ U = \sum_{ij}Y_{ij}\left(c^{\dagger}_{i\uparrow}c^{\dagger}_{i\downarrow}c_{j\downarrow}c_{j\uparrow} + c^{\dagger}_{j\uparrow}c^{\dagger}_{j\downarrow}c_{i\downarrow}c_{i\uparrow} \right) $$

Using nearest neighbour
$$Y_{ij} = \delta_{i,j+1} + \delta_{i+1,j}$$

In [17]:
#Nearest Neihgbour
def Y(i,j,N):
    if j == np.mod(i+1,N):
        return 1
    elif i == np.mod(j+1,N):
        return 1
    else:
        return 0

In [18]:
def U_Simultaneous(N):
    label_0 = ['I' for i in range(N)]
    l0 = ''.join(label_0)
    utp = O(l0 + l0)*0
    for i in range(0,N):
        for j in range(0,N):
            cdiu = cd(0,i,N)
            cdid = cd(1,i,N)
            cjd = c(1,j,N)
            cju = c(0,j,N)
            cdju = cd(0,j,N)
            cdjd = cd(1,j,N)
            cid = c(1,i,N)
            ciu = c(0,i,N)
            utp = utp + Y(i,j,N)*Mdot([cdiu,cdid,cjd,cju])+ Y(i,j,N)*Mdot([cdju,cdjd,cid,ciu])                                   
    return utp

In [19]:
print_matrix(U_Simultaneous(2))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### General Interaction

Here we have all to all interactions.  

$$ \hat{U} = \sum_{\sigma \sigma'}\sum_{ijkl} J_{(\sigma\sigma'ijkl)} c^{\dagger}_{i\sigma}c^{\dagger}_{j\sigma'}c_{k\sigma'}c_{l\sigma}$$

Let the interaction strength be given by*
$$ J_{(\sigma\sigma'ijkl)} = \frac{|i-l|+|j-l|+|\sigma - \sigma'|}{|i-j||k-l| + |\sigma - \sigma'|} $$

*just made it up based it becoming u when either $k=i$ and $j=l$ or $i=l$ and $j=k$.  Should check the real form.

In [28]:
def J(s,ss,i,j,k,l):
    return (abs(i-l)+abs(j-l)+abs(s-ss))/(abs(i-j)*abs(k-l)+abs(s-ss) + 0.001)

def U_General(N):
    label_0 = ['I' for i in range(N)]
    l0 = ''.join(label_0)
    utp = O(l0 + l0)*0
    for i in range(0,N):
        for j in range(0,N):
            for k in range(0,N):
                for l in range(0,N):
                    for s in range(0,2):
                        for ss in range(0,2):
                            cdis = cd(s,i,N)
                            cdjss = cd(ss,j,N)
                            ckss = c(ss,k,N)
                            cls = c(s,l,N)
                            #print(s,ss,i,j,k,l)
                            utp = utp + J(s,ss,i,j,k,l)*Mdot([cdis,cdjss,ckss,cls])                                
    return utp






In [29]:
print_matrix(U_General(2))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,4.0,6.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0


### Just so we have it I will print the kinetic part as well


$$ \hat{K} = \sum_{\sigma}\sum_{i} (c^{\dagger}_{i\sigma}c_{i+1\sigma'}+c^{\dagger}_{i+1\sigma'}c_{i\sigma})$$


In [37]:
def Kinetic(N):
    label_0 = ['I' for i in range(N)]
    l0 = ''.join(label_0)
    utp = O(l0 + l0)*0
    for i in range(0,N):
        for s in range(0,2):
            cd1 = cd(s,i,N)
            cd2 = cd(s,np.mod(i+1,N),N)
            c1 = c(s,i,N)
            c2 = c(s,np.mod(i+1,N),N)
            #print(s,ss,i,j,k,l)
            utp = utp + Mdot([cd1,c2]) + Mdot([cd2,c1])                                
    return utp


In [38]:
print_matrix(Kinetic(2))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0


## Fix number of spins

### Finding spin states

In [21]:
#Creates all states with N fermions and S orbitals
def Generate_States(N,S):
    s = [i for i in range(0,N)]
    psi_l = []
    while s[0] < S-N:
        ### create the state and store it
        psi = 0
        for i in s:
            psi += 2**i
        psi_l.append(psi)
        ###
        ###Update the particle locations
        exit = 0
        i = len(s)-1
        while exit == 0:
            if s[i] < S-len(s)+i:
                s[i] += 1
                for j in range(i+1,len(s)):
                    s[j] = s[j-1]+1
                exit = 1
            else:
                i -= 1
        ###
    ###Create the finale state
    psi = 0
    for i in s:
        psi += 2**i
    psi_l.append(psi)
    ###
    return psi_l

In [22]:
states = Generate_States(2,4)
for i in range(len(states)):
    print(bi(states[i],4))

0011
0101
1001
0110
1010
1100


In [23]:
#Given the states spinless states this finds the spinful states for N-sites
def Generate_Full_States(states,N):
    states_full = []
    for i in range(0,len(states)):
        for j in range(0,len(states)):
            states_full.append(states[i]*2**N+states[j])
    return states_full

In [24]:
states = Generate_States(2,4)
states_full = Generate_Full_States(states,4)
#for i in range(len(states_full)):
    #print(bi(states_full[i],8))

In [25]:
#Generates the basis vectors for N-sites
def basis_vectors(states,N):
    Nc = len(states)
    vecs = []
    for c in states:
        vec = [0 for i in range(2**N)]
        vec[c] = 1
        vecs.append(vec)
    return vecs


In [26]:
vecs = basis_vectors(states_full,8)

### Kinetic for 4-sites, 2-up, 2-down

In [39]:
Ki = Kinetic(4)

In [40]:
#Generate the spin restricted matrix from the full matrix
Kir = [[0 for i in range(0,len(vecs))] for j in range(len(vecs))]
for i in range(len(vecs)):
    for j in range(len(vecs)):
        Kir[i][j] = Mdot([vecs[i],Ki,vecs[j]])

In [41]:
print_matrix(Kir)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
0,0.0,1.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,1.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0
7,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0
8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0
9,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0


### General U for 4-sites, 2-up, 2-down

In [30]:
Ug = U_General(4)

In [31]:
#Generate the spin restricted matrix from the full matrix
Ugr = [[0 for i in range(0,len(vecs))] for j in range(len(vecs))]
for i in range(len(vecs)):
    for j in range(len(vecs)):
        Ugr[i][j] = Mdot([vecs[i],Ug,vecs[j]])

In [32]:
print_matrix(Ugr)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
0,8.0,6.0,8.0,-9.0,-10.7,0.0,6.0,6.0,8.0,-8.0,-5.0,0.0,8.0,8.0,10.0,-5.0,-12.0,0.0,-9.0,-8.0,-5.0,10.0,12.0,0.0,-10.7,-5.0,-12.0,12.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6.0,7.2,3.1,8.0,0.0,-10.7,2.0,6.0,3.0,3.0,0.0,-5.0,2.0,5.0,8.0,2.7,0.0,-12.0,-2.0,-12.0,-2.7,-8.0,0.0,12.0,-1.6,-10.7,-10.0,-10.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8.0,3.1,6.0,0.0,8.0,9.0,2.0,6.0,4.0,0.0,3.0,8.0,1.2,2.0,8.0,0.0,2.7,5.0,-2.0,-8.0,-8.0,0.0,-8.0,-10.0,-1.1,-2.0,-16.0,0.0,-10.0,-12.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-9.0,8.0,0.0,7.2,3.1,-8.0,-2.0,6.0,0.0,8.0,3.0,-8.0,-2.0,8.0,0.0,9.0,8.0,-10.0,1.2,-2.0,0.0,-9.0,-2.7,5.0,1.1,-2.0,0.0,-5.3,-10.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-10.7,0.0,8.0,3.1,6.0,6.0,-1.6,0.0,6.0,6.0,6.0,6.0,-1.1,0.0,8.0,2.0,12.0,8.0,1.1,0.0,-2.0,-8.0,-5.0,-8.0,0.8,0.0,-2.0,-2.0,-10.7,-5.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,-10.7,9.0,-8.0,6.0,5.2,0.0,-1.6,2.0,-2.0,2.0,6.0,0.0,-1.1,2.0,-1.2,2.0,9.0,0.0,1.1,-1.2,2.0,-2.0,-8.0,0.0,0.8,-1.1,1.1,-1.6,-10.7,0.0,0.0,0.0,0.0,0.0,0.0
6,6.0,2.0,2.0,-2.0,-1.6,0.0,7.2,6.0,5.0,-12.0,-10.7,0.0,3.1,3.0,8.0,-2.7,-10.0,0.0,8.0,3.0,2.7,-8.0,-10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.7,-5.0,-12.0,12.0,14.0,0.0
7,6.0,6.0,6.0,6.0,0.0,-1.6,6.0,6.4,5.1,6.0,0.0,-10.7,6.0,5.1,6.0,1.6,0.0,-10.0,6.0,6.0,1.6,6.0,0.0,-10.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.6,-10.7,-10.0,-10.0,0.0,14.0
8,8.0,3.0,4.0,0.0,6.0,2.0,5.0,5.1,6.0,0.0,6.0,12.0,2.0,2.0,5.1,0.0,1.6,2.7,8.0,2.0,5.1,0.0,6.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.1,-2.0,-16.0,0.0,-10.0,-12.0
9,-8.0,3.0,0.0,8.0,6.0,-2.0,-12.0,6.0,0.0,7.2,5.1,-5.0,-8.0,2.0,0.0,6.0,6.0,-8.0,-2.0,2.0,0.0,6.0,1.6,-2.7,0.0,0.0,0.0,0.0,0.0,0.0,1.1,-2.0,0.0,-5.3,-10.0,12.0


In [31]:
# For the s = ss transitions, ijkl = 0123 gets canceled by 1023 ect...
s=1
ss=1
i=1
j=0
k=2
l=3
N=4
utst = J(s,ss,i,j,k,l)*Mdot([cd(s,i,N),cd(ss,j,N),c(ss,k,N),c(s,l,N)])
print(J(s,ss,i,j,k,l))

Nc=len(vecs)
b=5
bb=0
print(bi(states_full[b],8))
print(bi(states_full[bb],8))
Mdot([vecs[b],utst,vecs[bb]])

4.9950049950049955
00111100
00110011


(4.9950049950049955+0j)

### Simultaneous Hopping for 4-sites, 2-up, 2-down

In [32]:
Us = U_Simultaneous(4)

In [33]:
#Generate the spin restricted matrix from the full matrix
Usr = [[0 for i in range(0,len(vecs))] for j in range(len(vecs))]
for i in range(len(vecs)):
    for j in range(len(vecs)):
        Usr[i][j] = Mdot([vecs[i],Us,vecs[j]])

In [34]:
print_matrix(Usr)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
7,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
print(bi(states_full[4],8))
print(bi(states_full[11],8))

00111010
01011100


### Coulomb-Assisted Hopping for 4-sites, 2-up, 2-down

In [36]:
Ua = U_Assisted(4)

In [37]:
#Generate the spin restricted matrix from the full matrix
Uar = [[0 for i in range(0,len(vecs))] for j in range(len(vecs))]
for i in range(len(vecs)):
    for j in range(len(vecs)):
        Uar[i][j] = Mdot([vecs[i],Ua,vecs[j]])

In [38]:
print_matrix(Uar)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
0,0.0,1.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,2.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0
7,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0
9,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
print(bi(states_full[2],8))
print(bi(states_full[26],8))

00111001
10101001


### Exchange Interaction for 4-sites, 2-up, 2-down

In [40]:
Ue = U_Exchange(4)

In [41]:
#Generate the spin restricted matrix from the full matrix
Uer = [[0 for i in range(0,len(vecs))] for j in range(len(vecs))]
for i in range(len(vecs)):
    for j in range(len(vecs)):
        Uer[i][j] = Mdot([vecs[i],Ue,vecs[j]])

In [42]:
print_matrix(Uer)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,-2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,2.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
print(bi(states_full[8],8))
print(bi(states_full[13],8))

01011001
10010101


### Hartree Interaction for 4-sites, 2-up, 2-down

In [44]:
Uh = U_Hartree(4)

In [45]:
#Generate the spin restricted matrix from the full matrix
Uhr = [[0 for i in range(0,len(vecs))] for j in range(len(vecs))]
for i in range(len(vecs)):
    for j in range(len(vecs)):
        Uhr[i][j] = Mdot([vecs[i],Uh,vecs[j]])

In [46]:
print_matrix(Uhr)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### On-Site Interaction for 4-sites, 2-up, 2-down

In [47]:
Uo = U_OnSite(1,4)

In [48]:
#Generate the spin restricted matrix from the full matrix
Uor = [[0 for i in range(0,len(vecs))] for j in range(len(vecs))]
for i in range(len(vecs)):
    for j in range(len(vecs)):
        Uor[i][j] = Mdot([vecs[i],Uo,vecs[j]])

In [49]:
print_matrix(Uor)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
print(latex_matrix(Kir))

\setcounter{MaxMatrixCols}{36} \begin{equation} \begin{pmatrix} 0.0 & 1.0 & 0.0 & 0.0 & -1.0 & 0.0 & 1.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & -1.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 \\1.0 & 0.0 & 1.0 & 1.0 & 0.0 & -1.0 & 0.0 & 1.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & -1.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 \\0.0 & 1.0 & 0.0 & 0.0 & 1.0 & 0.0 & 0.0 & 0.0 & 1.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & -1.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 \\0.0 & 1.0 & 0.0 & 0.0 & 1.0 & 0.0 & 0.0 & 0.0 & 0.0 & 1.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & -1.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 \\-1.0 & 0.0 & 1.0 & 1.0 & 0.0 & 1.0 & 0.0 & 0.0 & 0.0 & 0.0 & 1.0 &

In [51]:
pwd

'/Users/stenger/Documents/Research/Hubbard_symmetries'

In [58]:
basis = [bi(s,8) for s in states_full]

In [85]:
def latex_vvector(V):
    ltp = "\setcounter{MaxMatrixCols}{" + str(len(V)) +"} \\begin{equation} \\begin{pmatrix} "
    for e in V:
        ltp = ltp + '|' + e + '>' + " \\\\ "
    ltp = ltp[:-3]
    ltp = ltp + "\\end{pmatrix} \\end{equation}"
    return ltp

def latex_hvector(V):
    ltp = "\setcounter{MaxMatrixCols}{" + str(len(V)) +"} \\begin{equation} \\begin{pmatrix} "
    for e in V:
        ltp = ltp + '|' + e + '>' + " & "
    ltp = ltp[:-2]
    ltp = ltp + "\\end{pmatrix} \\end{equation}"
    return ltp

In [86]:
print(latex_vvector(basis))

\setcounter{MaxMatrixCols}{36} \begin{equation} \begin{pmatrix} |00110011> \\ |00110101> \\ |00111001> \\ |00110110> \\ |00111010> \\ |00111100> \\ |01010011> \\ |01010101> \\ |01011001> \\ |01010110> \\ |01011010> \\ |01011100> \\ |10010011> \\ |10010101> \\ |10011001> \\ |10010110> \\ |10011010> \\ |10011100> \\ |01100011> \\ |01100101> \\ |01101001> \\ |01100110> \\ |01101010> \\ |01101100> \\ |10100011> \\ |10100101> \\ |10101001> \\ |10100110> \\ |10101010> \\ |10101100> \\ |11000011> \\ |11000101> \\ |11001001> \\ |11000110> \\ |11001010> \\ |11001100> \end{pmatrix} \end{equation}


In [72]:
print(latex_hvector(basis))

\setcounter{MaxMatrixCols}{36} \begin{equation} \begin{pmatrix} |00110011> & |00110101> & |00111001> & |00110110> & |00111010> & |00111100> & |01010011> & |01010101> & |01011001> & |01010110> & |01011010> & |01011100> & |10010011> & |10010101> & |10011001> & |10010110> & |10011010> & |10011100> & |01100011> & |01100101> & |01101001> & |01100110> & |01101010> & |01101100> & |10100011> & |10100101> & |10101001> & |10100110> & |10101010> & |10101100> & |11000011> & |11000101> & |11001001> & |11000110> & |11001010> & |11001100> \end{pmatrix} \end{equation}
